In [1]:
cd("/g/data/v45/nc3020/Oceananigans-Tutorial-CFP/near-global/")

ENV["JULIA_DEPOT_PATH"] = "/g/data/v45/nc3020/.julia:/share/julia/site/"
ENV["JULIA_LOAD_PATH"] = "@:@v#.#:@stdlib:@site"

using Pkg; Pkg.activate(@__DIR__);

  Activating project at `/g/data/v45/nc3020/Oceananigans-Tutorial-CFP/near-global`


In [2]:
using ClimaOcean.NearGlobalSimulations: quarter_degree_near_global_simulation

using Oceananigans
using Oceananigans.Units
using Oceananigans.Utils: WallTimeInterval
using Oceananigans.BuoyancyModels: buoyancy
using Oceananigans.Models.HydrostaticFreeSurfaceModels: VerticalVorticityField
using JLD2

[ Info: Oceananigans will use 12 threads
┌ Warning: Multiple CUDA environment variables set to different values: /g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10, /apps/cuda/11.4.1
└ @ CUDA.Deps /g/data/v45/nc3020/.julia/packages/CUDA/BbliS/deps/discovery.jl:268


In [3]:
boundary_layer_turbulence_closure = RiBasedVerticalDiffusivity()

┌ Warning: RiBasedVerticalDiffusivity is an experimental turbulence closure that 
│ is unvalidated and whose default parameters are not calibrated for 
│ realistic ocean conditions or for use in a three-dimensional 
│ simulation. Use with caution and report bugs and problems with physics 
│ to https://github.com/CliMA/Oceananigans.jl/issues.
└ @ Oceananigans.TurbulenceClosures /g/data/v45/nc3020/.julia/packages/Oceananigans/KTw3g/src/TurbulenceClosures/turbulence_closure_implementations/ri_based_vertical_diffusivity.jl:83


RiBasedVerticalDiffusivity{VerticallyImplicitTimeDiscretization}
├── Ri_dependent_tapering: HyperbolicTangentRiDependentTapering
├── κ₀: 0.5
├── κᶜᵃ: 1.7
├── Cᵉⁿ: 0.1
├── Cᵃᵛ: 0.6
├── Ri₀: 0.1
└── Riᵟ: 0.4

In [8]:
start_time = 0days
stop_time  = start_time + 30days

architecture = GPU()

simulation = quarter_degree_near_global_simulation(architecture;
                                                   start_time, stop_time, boundary_layer_turbulence_closure)

[ Info: Reading initial conditions...
[ Info: ... read initial conditions (448.884 ms)
[ Info: Reading boundary conditions...
[ Info: ... read boundary conditions (402.984 ms)
┌ Info: Created 1440×600×48 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo:
│ ├── immersed_boundary: GridFittedBottom(min(h)=-1.05e+04, max(h)=1.00e+02)
│ ├── underlying_grid: 1440×600×48 LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo and with precomputed metrics
│ ├── longitude: Periodic λ ∈ [-180.0, 180.0) regularly spaced with Δλ=0.25
│ ├── latitude:  Bounded  φ ∈ [-75.0, 75.0]   regularly spaced with Δφ=0.25
└ └── z:         Bounded  z ∈ [-5244.5, 0.0]  variably spaced with min(Δz)=10.0, max(Δz)=410.5
┌ Info: ... built HydrostaticFreeSurfaceModel{GPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
│ ├── grid: 1440×600×48 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo
│ ├── timestepper: QuasiAdamsBash

Simulation of HydrostaticFreeSurfaceModel{GPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 30 days
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 5 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   └── progress => Callback of (::ClimaOcean.NearGlobalSimulations.var"#progress#9"{Vector{UInt64}}) on IterationInterval(10)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [9]:
# Define output
slices_save_interval = 6hours
fields_save_interval = 5days
Nx, Ny, Nz = size(simulation.model.grid)

dir = pwd() * "/output"

closure_name = typeof(boundary_layer_turbulence_closure).name.wrapper
output_prefix = "near_global_$(Nx)_$(Ny)_$(Nz)_$closure_name"

"near_global_1440_600_48_RiBasedVerticalDiffusivity"

In [10]:
model = simulation.model

simulation.output_writers[:checkpointer] = Checkpointer(model; dir,
                                                        prefix = output_prefix * "_checkpointer",
                                                        schedule = WallTimeInterval(10minutes),
                                                        cleanup = true,
                                                        overwrite_existing = true)

simulation.output_writers[:fields] = JLD2OutputWriter(model, merge(model.velocities, model.tracers); dir,
                                                      schedule = TimeInterval(slices_save_interval),
                                                      filename = output_prefix * "_fields",
                                                      with_halos = true,
                                                      overwrite_existing = true)

slice_indices = [(:, :, Nz), (:, :, Nz-10)]
output_names = [:surface, :near_surface]

for n = 1:2
    indices = slice_indices[n]

    outputs = Dict()

    for name in keys(model.tracers)
        c = model.tracers[name]
        outputs[name] = Field(c; indices)
    end

    outputs[:u] = Field(model.velocities.u; indices)
    outputs[:v] = Field(model.velocities.v; indices)
    outputs[:w] = Field(model.velocities.w; indices)
    outputs[:η] = model.free_surface.η
    outputs[:ζ] = VerticalVorticityField(model.grid, model.velocities; indices)

    name = output_names[n]
    simulation.output_writers[name] = JLD2OutputWriter(model, outputs; dir,
                                                       schedule = TimeInterval(slices_save_interval),
                                                       filename = output_prefix * "_fields_$name",
                                                       with_halos = true,
                                                       overwrite_existing = true)
end

In [11]:
# start with small timestep

spinup = 2days
simulation.Δt = 2minute
stop_time = simulation.stop_time
simulation.stop_time = time(simulation) + spinup

@info "Running spinup simulation with Δt = $(prettytime(simulation.Δt)) until t = $(prettytime(simulation.stop_time))"
run!(simulation)

# increase timestep a bit

spinup = 2days
simulation.Δt = 5minute
simulation.stop_time = time(simulation) + spinup

@info "Running some more spinup simulation with Δt = $(prettytime(simulation.Δt)) until t = $(prettytime(simulation.stop_time))"
run!(simulation)



# now increase timestep

simulation.stop_time = start_time + stop_time - spinup
simulation.Δt = 10minutes

@info "Running simulation with Δt = $(prettytime(simulation.Δt)) until t = $(prettytime(spinup))"
run!(simulation)

@info "Simulation took $(prettytime(simulation.run_wall_time))."

[ Info: Running spinup simulation with Δt = 2 minutes until t = 2 days
[ Info: Initializing simulation...
[ Info: Time:    0 seconds, iteration: 0, max(|u|): 0.00e+00 ms⁻¹, wmax: 0.00e+00, loc: (1, 1, 1), wall time: 549.505 ms
[ Info:     ... simulation initialization complete (20.415 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (342.898 ms).
[ Info: Time:   20 minutes, iteration: 10, max(|u|): 1.24e-01 ms⁻¹, wmax: 2.12e-03, loc: (1411, 85, 15), wall time: 22.943 seconds
[ Info: Time:   40 minutes, iteration: 20, max(|u|): 2.88e-01 ms⁻¹, wmax: 3.74e-03, loc: (1404, 82, 10), wall time: 2.762 seconds
[ Info: Time:       1 hour, iteration: 30, max(|u|): 4.42e-01 ms⁻¹, wmax: 5.22e-03, loc: (1411, 85, 15), wall time: 2.760 seconds
[ Info: Time:  1.333 hours, iteration: 40, max(|u|): 4.71e-01 ms⁻¹, wmax: 6.50e-03, loc: (1411, 85, 15), wall time: 2.758 seconds
[ Info: Time:  1.667 hours, iteration: 50, max(|u|): 6.68e-01 ms⁻¹, wmax: 7.68e-03, loc:

KeyError: KeyError: key "usage_request" not found

[ Info: Time:  22.778 days, iteration: 4720, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.50e-02, loc: (129, 528, 5), wall time: 30.982 seconds


KeyError: KeyError: key "usage_request" not found

[ Info: Time:  22.847 days, iteration: 4730, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.51e-02, loc: (129, 528, 5), wall time: 3.498 seconds
[ Info: Time:  22.917 days, iteration: 4740, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.58e-02, loc: (1129, 42, 41), wall time: 3.478 seconds
[ Info: Time:  22.986 days, iteration: 4750, max(|u|): 2.44e+01 ms⁻¹, wmax: 3.36e-02, loc: (1129, 42, 41), wall time: 3.443 seconds
[ Info: Time:  23.056 days, iteration: 4760, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.90e-02, loc: (1129, 42, 42), wall time: 8.919 seconds


...

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

[ Info: Time:  23.542 days, iteration: 4830, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.59e-02, loc: (129, 528, 5), wall time: 11.068 seconds
[ Info: Time:  23.611 days, iteration: 4840, max(|u|): 2.44e+01 ms⁻¹, wmax: 2.60e-02, loc: (129, 528, 5), wall time: 3.516 seconds
[ Info: Time:  23.681 days, iteration: 4850, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.78e-02, loc: (1129, 44, 29), wall time: 3.489 seconds
[ Info: Time:  23.750 days, iteration: 4860, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.61e-02, loc: (129, 528, 5), wall time: 3.496 seconds
[ Info: Time:  23.819 days, iteration: 4870, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.62e-02, loc: (129, 528, 5), wall time: 13.156 seconds
[ Info: Time:  23.889 days, iteration: 4880, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.63e-02, loc: (129, 528, 5), wall time: 3.446 seconds
[ Info: Time:  23.958 days, iteration: 4890, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.64e-02, loc: (129, 528, 5), wall time: 3.433 seconds
[ Info: Time:  24.028 days, iteration: 4900, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.65

KeyError: KeyError: key "usage_request" not found

[ Info: Time:  26.458 days, iteration: 5250, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.91e-02, loc: (129, 528, 5), wall time: 3.450 seconds
[ Info: Time:  26.528 days, iteration: 5260, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.93e-02, loc: (129, 528, 5), wall time: 12.057 seconds
[ Info: Time:  26.597 days, iteration: 5270, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.94e-02, loc: (129, 528, 5), wall time: 3.441 seconds
[ Info: Time:  26.667 days, iteration: 5280, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.95e-02, loc: (129, 528, 5), wall time: 3.437 seconds
[ Info: Time:  26.736 days, iteration: 5290, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.96e-02, loc: (129, 528, 5), wall time: 3.445 seconds
[ Info: Time:  26.806 days, iteration: 5300, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.96e-02, loc: (129, 528, 5), wall time: 12.180 seconds
[ Info: Time:  26.875 days, iteration: 5310, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.97e-02, loc: (129, 528, 5), wall time: 3.447 seconds
[ Info: Time:  26.944 days, iteration: 5320, max(|u|): 2.43e+01 ms⁻¹, wmax: 2.98e

In [12]:
saved_output_filename = output_prefix * "_fields_surface" * ".jld2"

"near_global_1440_600_48_RiBasedVerticalDiffusivity_fields_surface.jld2"

In [13]:
backend = OnDisk()
η_t = FieldTimeSeries("output/" * saved_output_filename, "η"; backend)
ζ_t = FieldTimeSeries("output/" * saved_output_filename, "ζ"; backend)

times = ζ_t.times

113-element Vector{Float64}:
      0.0
  21600.0
  43200.0
  64800.0
  86400.0
 108000.0
 129600.0
 151200.0
 172800.0
 194400.0
 216000.0
 237600.0
 259200.0
      ⋮
      2.1816e6
      2.2032e6
      2.2248e6
      2.2464e6
      2.268e6
      2.2896e6
      2.3112e6
      2.3328e6
      2.3544e6
      2.376e6
      2.3976e6
      2.4192e6

In [14]:
λη, φη, zη = nodes(η_t[1])
λζ, φζ, zζ = nodes(ζ_t[1])

([-180.0, -179.75, -179.5, -179.25, -179.0, -178.75, -178.5, -178.25, -178.0, -177.75  …  177.5, 177.75, 178.0, 178.25, 178.5, 178.75, 179.0, 179.25, 179.5, 179.75], [-75.0, -74.75, -74.5, -74.25, -74.0, -73.75, -73.5, -73.25, -73.0, -72.75  …  72.75, 73.0, 73.25, 73.5, 73.75, 74.0, 74.25, 74.5, 74.75, 75.0], [-5039.25, -4640.25, -4264.25, -3911.25, -3581.25, -3274.25, -2990.25, -2729.25, -2491.25, -2276.225  …  -95.095, -85.025, -75.005, -65.0, -55.0, -45.0, -35.0, -25.0, -15.0, -5.0])

In [15]:
# using Oceananigans.ImmersedBoundaries: mask_immersed_field!

# function mask_and_get_interior(φ_t, n)
#     mask_immersed_field!(φ_t[n], NaN)
#     return interior(φ_t[n], :, :, 1)
# end

mask_and_get_interior (generic function with 1 method)

In [16]:
use CairoMakie

LoadError: syntax: extra token "CairoMakie" after end of expression

In [17]:
n = Observable(1)

using Printf
title = @lift @sprintf("t = %1.2f days", round(times[$n]/day, digits=2))

ηₙ = @lift mask_and_get_interior(η_t, $n)
ζₙ = @lift mask_and_get_interior(ζ_t, $n)

axis_kwargs = (titlesize = 20,)

ηlim   = maximum(abs, η_t[end])
ζlim   = maximum(abs, ζ_t[end])

fig = Figure(resolution = (1000, 900))

ax_η = Axis(fig[2, 1];
            title = "sea surface height", axis_kwargs...)

ax_ζ = Axis(fig[3, 1];
            title = "vorticity", axis_kwargs...)

fig[1, :] = Label(fig, title, fontsize=24, tellwidth=false)

hm_η = heatmap!(ax_η, λη, φη, ηₙ;
                colorrange = (-ηlim, ηlim),
                colormap = :balance)
Colorbar(fig[2, 2], hm_η)

hm_ζ = heatmap!(ax_ζ, λζ, φζ, ζₙ;
                colorrange = (-ζlim, ζlim),
                colormap = :balance)
Colorbar(fig[3, 2], hm_ζ)

fig

LoadError: UndefVarError: Observable not defined

In [ ]:
@info "Making a movie from saved data..."

frames = 1:length(times)

CairoMakie.record(fig, "quarter_degree.mp4", frames, framerate=24) do i
    msg = string("Plotting frame ", i, " of ", frames[end])
    print(msg * " \r")
    n[] = i
end